## Dependencies

In [ ]:
### Installing dependencies on Google Colab
!pip install gradio
!pip install langchain
!pip install google-generativeai
!pip install langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


### Loading core dependencies

In [ ]:
import io
import base64
import gradio as gr
from PIL import Image
from typing import Tuple
from functools import wraps

### Loading LangChain and GenAI modules

In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

## Application body

### API Decorator and Input Validator

In [ ]:
def handle_api_errors(func):
    """Decorator to handle common API errors gracefully."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            error_msg = str(e)
            if "api key" in error_msg.lower():
                return "API key error: Please check your Gemini API key"
            elif "quota" in error_msg.lower():
                return "API quota exceeded: Please check your usage limits"
            else:
                return f"Error: {error_msg[:100]}..."
    return wrapper

def validate_inputs(func):
    """Decorator to validate inputs before processing."""
    @wraps(func)
    def wrapper(input_data, api_key, *args, **kwargs):
        if not api_key or not api_key.strip():
            return "Error: API key is required"
        if not input_data:
            return "Error: Input data is required"
        return func(input_data, api_key, *args, **kwargs)
    return wrapper

### Agents and Tools

In [ ]:
@handle_api_errors
@validate_inputs
def identify_plant_tool(image_base64: str, api_key: str) -> str:
    """
    Core plant identification agent using Gemini Vision.

    Args:
        image_base64: Base64 encoded plant image
        api_key: Google Gemini API key

    Returns:
        Raw identification data for processing
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')

    image_bytes = base64.b64decode(image_base64)
    image = Image.open(io.BytesIO(image_bytes))

    prompt = """
    Analyze this plant image and provide identification data:

    COMMON NAME: [Name]
    SCIENTIFIC NAME: [Genus species]
    FAMILY: [Plant family]
    CONFIDENCE: [High/Medium/Low]

    KEY FEATURES:
    • [Feature 1]
    • [Feature 2]
    • [Feature 3]

    DESCRIPTION:
    [Brief botanical description]
    """

    response = model.generate_content([prompt, image])
    return response.text

@handle_api_errors
@validate_inputs
def research_plant_tool(plant_name: str, api_key: str) -> str:
    """
    Botanical research agent for comprehensive plant data.

    Args:
        plant_name: Name of plant to research
        api_key: Google Gemini API key

    Returns:
        Raw research data for processing
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')

    prompt = f"""
    Research comprehensive botanical information for: {plant_name}

    BOTANICAL DETAILS:
    • Type: [Classification]
    • Height: [Size range]
    • Native Region: [Geographic origin]
    • Habitat: [Natural environment]

    GROWING CONDITIONS:
    • Light: [Requirements]
    • Water: [Needs]
    • Soil: [Preferences]
    • Temperature: [Range]
    • Humidity: [Preferences]

    INTERESTING FACTS:
    • [Notable characteristic 1]
    • [Notable characteristic 2]
    • [Notable characteristic 3]

    USES:
    • [Primary applications]
    """

    response = model.generate_content(prompt)
    return response.text

@handle_api_errors
@validate_inputs
def get_care_instructions_tool(plant_name: str, api_key: str) -> str:
    """
    Plant care specialist agent for cultivation guidance.

    Args:
        plant_name: Name of plant
        api_key: Google Gemini API key

    Returns:
        Raw care instruction data
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')

    prompt = f"""
    Generate cultivation guidelines for: {plant_name}

    CARE DIFFICULTY: [Easy/Medium/Hard]

    WATERING:
    • Frequency: [Schedule]
    • Amount: [Quantity]
    • Seasonal adjustments: [Variations]

    LIGHTING:
    • Indoor requirements: [Specifications]
    • Outdoor needs: [Conditions]

    SOIL & NUTRITION:
    • Soil type: [Requirements]
    • pH preference: [Range]
    • Fertilizer: [Type and schedule]

    COMMON ISSUES:
    • [Problem and solution]
    • [Problem and solution]

    PROPAGATION:
    • Method: [Technique]
    • Timing: [Best season]
    """

    response = model.generate_content(prompt)
    return response.text

@handle_api_errors
@validate_inputs
def get_geographic_data_tool(plant_name: str, api_key: str) -> str:
    """
    Geographic distribution specialist agent.

    Args:
        plant_name: Name of plant
        api_key: Google Gemini API key

    Returns:
        Raw geographic distribution data
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')

    prompt = f"""
    Compile geographic and climate data for: {plant_name}

    NATIVE RANGE:
    • Original habitat: [Location]
    • Climate zones: [Classifications]

    CURRENT DISTRIBUTION:
    • Cultivation regions: [Areas]
    • Hardiness zones: [USDA zones]
    • Invasive status: [If applicable]

    SEASONAL PATTERNS:
    • Growing season: [Period]
    • Flowering time: [Schedule]
    • Dormancy: [If applicable]

    CLIMATE REQUIREMENTS:
    • Temperature range: [Optimal conditions]
    • Precipitation: [Needs]
    • Elevation: [Preferences]
    """

    response = model.generate_content(prompt)
    return response.text

@handle_api_errors
@validate_inputs
def format_output_tool(raw_content: str, api_key: str) -> str:
    """
    Output formatting agent to clean and professionalize responses.

    Args:
        raw_content: Raw AI response content
        api_key: Google Gemini API key

    Returns:
        Clean, professional formatted output
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash')

    prompt = f"""
    Clean and format this botanical content for professional presentation:

    {raw_content}

    FORMATTING RULES:
    1. Remove conversational prefixes like "Okay, here's..." or "I'll analyze..."
    2. Remove redundant introductions and conclusions
    3. Start directly with the factual content
    4. Maintain all bullet points and structure
    5. Keep scientific accuracy intact
    6. Ensure professional, reference-quality tone
    7. Remove any meta-commentary about the analysis process

    Return ONLY the cleaned, formatted content - no additional text.
    """

    response = model.generate_content(prompt)
    return response.text.strip()

    def analyze_plant(self, image_base64: str) -> dict:
        """
        Execute complete plant analysis workflow with professional formatting.

        Args:
            image_base64: Base64 encoded plant image

        Returns:
            Dictionary with formatted analysis results
        """
        try:
            executor = AgentExecutor(
                agent=self.agent,
                tools=self.tools,
                memory=self.memory,
                verbose=False,
                max_iterations=15,
                handle_parsing_errors=True
            )

            query = f"Analyze this plant image and provide comprehensive information: {image_base64[:50]}..."
            result = executor.invoke({"input": query})

            return {"success": True, "result": result["output"]}

        except Exception as e:
            return {"success": False, "error": str(e)}

def validate_api_key(api_key: str) -> Tuple[bool, str]:
    """Validate Google Gemini API key functionality."""
    if not api_key or len(api_key.strip()) < 10:
        return False, "API key appears invalid"

    try:
        genai.configure(api_key=api_key.strip())
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content("Test")
        return True, "API key validated"
    except Exception as e:
        return False, f"Validation failed: {str(e)}"

def analyze_plant_comprehensive(image: Image.Image, api_key: str, progress=gr.Progress()) -> Tuple:
    """
    Main analysis function with LangChain agent coordination.

    Args:
        image: Plant image to analyze
        api_key: Google Gemini API key
        progress: Gradio progress tracker

    Returns:
        Tuple of analysis results
    """
    if not api_key or not api_key.strip():
        return (None, "Error: API key required", "", "", "", "")

    if image is None:
        return (None, "Upload plant image to begin", "", "", "", "")

    try:
        progress(0.1, desc="Validating credentials...")

        is_valid, message = validate_api_key(api_key)
        if not is_valid:
            return (None, f"API Error: {message}", "", "", "", "")

        progress(0.2, desc="Initializing analysis agents...")

        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        image_base64 = base64.b64encode(buffered.getvalue()).decode()

        progress(0.3, desc="Running identification...")

        # Step 1: Direct identification for plant name extraction
        identification = identify_plant_tool(image_base64, api_key.strip())

        plant_name = "Unknown Plant"
        if "COMMON NAME:" in identification:
            try:
                plant_name = identification.split("COMMON NAME:")[1].split("\n")[0].strip()
            except:
                plant_name = "Unknown Plant"

        progress(0.5, desc="Researching botanical data...")

        # Step 2: Research tasks
        detailed_info = research_plant_tool(plant_name, api_key.strip())

        progress(0.7, desc="Generating care instructions...")

        care_info = get_care_instructions_tool(plant_name, api_key.strip())

        progress(0.8, desc="Compiling geographic data...")

        geo_info = get_geographic_data_tool(plant_name, api_key.strip())

        progress(0.9, desc="Formatting outputs...")

        # Step 3: Clean all outputs with formatting agent
        if plant_name == 'Unknown Plant':
            clean_identification = "Plant name not found"
            clean_detailed = "Plant name not found"
            clean_care = "Plant name not found"
            clean_geo = "Plant name not found"
        else:
          clean_identification = format_output_tool(identification, api_key.strip())
          clean_detailed = format_output_tool(detailed_info, api_key.strip())
          clean_care = format_output_tool(care_info, api_key.strip())
          clean_geo = format_output_tool(geo_info, api_key.strip())

        progress(1.0, desc="Analysis complete")

        return (
            image,
            "Analysis complete",
            clean_identification,
            clean_detailed,
            clean_care,
            clean_geo
        )

    except Exception as e:
        error_msg = str(e)
        display_error = error_msg[:100] + "..." if len(error_msg) > 100 else error_msg
        return (None, f"ERROR: {display_error}", "", "", "", "")

### Interface

In [ ]:
def create_interface():
    """Create the main Gradio interface focused on LangChain functionality."""

    with gr.Blocks(
        title="Plant Identification System",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            background: linear-gradient(135deg, #1e293b 0%, #6495ED 100%);
            font-family: 'Segoe UI', 'Roboto', sans-serif;
            font-color: #10b981;
            font-size: 16px;
            min-height: 100vh;
        }

        .header-section {
            background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
            color: white;
            padding: 30px;
            text-align: center;
            border-radius: 12px;
            margin-bottom: 25px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }

        .header-section h1 {
            margin: 0;
            font-size: 2.5rem;
            font-weight: 700;
        }

        .header-section p {
            margin: 10px 0 0 0;
            font-size: 1.1rem;
            opacity: 0.9;
        }

        .api-section {
            background: cornflowerblue;
            padding: 25px;
            border-radius: 12px;
            box-shadow: 0 4px 24px rgba(0,0,0,0.08);
            border: 1px solid #e2e8f0;
            margin-bottom: 25px;
        }

        .main-container {
            background: gainsboro;
            border-radius: 16px;
            padding: 30px;
            border: 1px solid #e2e8f0;
        }

        .section-header {
            color: #1e293b !important;
            font-size: 1.2rem !important;
            font-weight: 600 !important;
            margin: 0 0 15px 0 !important;
            padding: 0 0 8px 0 !important;
            border-bottom: 2px solid #e2e8f0 !important;
        }

        .content-area {
            background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
            color: #10b981;
            border: none;
            border-radius: 12px;
            padding: 20px;
            font-family: 'SF Mono', 'Monaco', 'Cascadia Code', monospace;
            font-size: 14px;
            font-weight: 600;
            line-height: 1.6;
            min-height: 200px;
            text-shadow: 0 0 10px rgba(16, 185, 129, 0.8);
            white-space: pre-wrap;
            overflow-y: auto;
        }

        .status-display {
            background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
            color: #10b981;
            padding: 20px;
            font-family: 'SF Mono', 'Monaco', monospace;
            font-size: 14px;
            font-weight: 600;
            text-align: center;
            text-shadow: 0 0 10px rgba(16, 185, 129, 0.8);
        }

        .analyze-button {
            background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
            color: #10b981;
            border: none !important;
            padding: 16px 32px !important;
            border-radius: 12px !important;
            font-size: 1.1rem !important;
            font-weight: 600 !important;
            text-transform: uppercase !important;
            transition: all 0.2s ease !important;
            box-shadow: 0 8px 24px rgba(5, 150, 105, 0.3) !important;
            width: 100% !important;
            margin-top: 20px !important;
        }

        .analyze-button:hover {
            background: linear-gradient(135deg, #1e293b 0%, #334155 100%)) !important;
            transform: translateY(-2px) !important;
        }
        """
    ) as interface:

        gr.HTML("""
        <div class="header-section">
            <h1>🌿 Plant Identification System</h1>
            <p>LangChain-powered botanical analysis with professional formatting</p>
        </div>
        """)

        with gr.Column():
            gr.HTML("""
            <div class="api-section">
                <h3>🔑 Authentication Required</h3>
                <p>Enter your Google Gemini API key to enable the analysis agents</p>
                <a href="https://makersuite.google.com/app/apikey" target="_blank">→ Get your API key</a>
            </div>
            """)

            api_key = gr.Textbox(
                type="password",
                label="Google Gemini API Key",
                placeholder="Enter your API key here..."
            )

        with gr.Column(elem_classes="main-container"):

            with gr.Column(scale=1):
                    plant_image = gr.Image(
                        label="📸 Upload Plant Photo",
                        type="pil",
                        height=350
                    )

                    analyze_btn = gr.Button(
                        "Run Analysis Pipeline",
                        elem_classes="analyze-button"
                    )

            with gr.Column(scale=1):
                    status_output = gr.Markdown(
                        value="System Ready",
                        elem_classes="status-display"
                    )

            with gr.Row():
                with gr.Column():
                    gr.HTML("<h3 class='section-header'>🔍 Plant Identification</h3>")
                    identification_output = gr.Markdown(
                        value="",
                        elem_classes="content-area"
                    )

                with gr.Column():
                    gr.HTML("<h3 class='section-header'>📋 Botanical Research</h3>")
                    details_output = gr.Markdown(
                        value="",
                        elem_classes="content-area"
                    )

            with gr.Row():
                with gr.Column():
                    gr.HTML("<h3 class='section-header'>🌱 Care Instructions</h3>")
                    care_output = gr.Markdown(
                        value="",
                        elem_classes="content-area"
                    )

                with gr.Column():
                    gr.HTML("<h3 class='section-header'>🗺️ Geographic Distribution</h3>")
                    geography_output = gr.Markdown(
                        value="",
                        elem_classes="content-area"
                    )

        analyze_btn.click(
            fn=analyze_plant_comprehensive,
            inputs=[plant_image, api_key],
            outputs=[
                plant_image,
                status_output,
                identification_output,
                details_output,
                care_output,
                geography_output
            ]
        )

    return interface

## Application launch

In [ ]:
if __name__ == "__main__":
    print("🌿 Starting Plant Identification System...")

    interface = create_interface()
    interface.launch(share=True, debug=True)

🌿 Starting Plant Identification System...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef7024b1689066d9cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef7024b1689066d9cb.gradio.live
